In [2]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm
# np.random.seed(200)

In [3]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data",names=["parents","has_nurs","form","children","housing","finance","social","health","Class"])

In [7]:
train_df = df.sample(frac=0.8,random_state=200)
test_df = df.drop(train_df.index)

In [8]:
train_y = train_df['Class']
train_X = train_df.drop('Class',axis=1)
test_y = test_df['Class']
test_X = test_df.drop('Class',axis=1)

In [9]:
cols = train_X.columns
num_cols = train_X._get_numeric_data().columns
num_count = len(num_cols)
cat_cols = list(set(cols) - set(num_cols))
cat_count = len(cat_cols)
print("number of numerical attributes of Dataset = ",num_count ,"\nnumber of categorical attributes of Dataset = ",cat_count)

number of numerical attributes of Dataset =  0 
number of categorical attributes of Dataset =  8


In [4]:
column_names = list(df.columns)

unique_values_arr = []
for i in column_names :
  unique_values = df[i].unique()
  unique_values_arr.append(unique_values)
import warnings
warnings.filterwarnings("ignore")

unique_values_arr_y = unique_values_arr[column_names.index('Class')]
unique_values_arr_X = np.delete(unique_values_arr,column_names.index('Class'),axis=0)

In [10]:
column_names_X = list(train_X.columns)

In [ ]:
max_unique_values = 0
for i in column_names_X :
  unique_values = df[i].unique()
  if len(unique_values) >= max_unique_values  :
    max_unique_values  = len(unique_values)

In [ ]:
grouped = train_df.groupby('Class')

In [ ]:
count_Class =  train_df['Class'].value_counts()

In [ ]:
order = unique_values_arr_y
count_y = count_Class.reindex(order)

In [ ]:
def Accuracy_NB(epsilon):
  prob_cond_num = [[[0.0 for _ in range(len(unique_values_arr_y))] for _ in range(max_unique_values )] for _ in range(len(column_names_X))]
  for col in column_names_X:
    if df[col].dtype == 'object':
      s = 1
      for col in column_names_X:
        if train_df[col].dtype == 'object':
          for i in range(len(unique_values_arr_X[column_names_X.index(col)])):
            for k in range(len(unique_values_arr_y)):
              count = len(train_df[(train_df[col] == unique_values_arr_X[column_names_X.index(col)][i]) & (train_df['Class'] == unique_values_arr_y[k])])
              count = np.random.laplace(count, s/epsilon)
              count = max(count,0)
              prob = count/(count_y[unique_values_arr_y[k]])
              prob_cond_num[column_names_X.index(col)][i][np.where(unique_values_arr_y == unique_values_arr_y[k])[0][0]] = prob
    else:
      for i in unique_values_arr_y :
        df_class = grouped.get_group(i).drop('Class',axis=1)
        
        u = df_class[col].max()
        l = df_class[col].min()
        n = len(df_class)

        mean = df_class[col].mean()
        sensitivity_mean = (u-l)/(n+1)
        sf_mean = sensitivity_mean/epsilon
        std = df_class[col].std()
        sensitivity_var = n**0.5* (u-l)/(n+1)
        sf_std = sensitivity_var/epsilon
        for j in range(len(unique_values_arr_X[column_names_X.index(col)])):
          x_1 = unique_values_arr_X[column_names_X.index(col)][j]
          x = np.random.laplace(mean,sf_mean)+np.random.laplace(std,sf_std)
          prob= 1/np.sqrt(2*np.pi)/std*np.exp(-((x-mean)**2)/2/std**2)
          prob_cond_num[column_names.index(col)][j][np.where(unique_values_arr_y == i)[0][0]] = prob
  Prior_prob = []
  noise_1 = abs(np.random.laplace(0,1))
  for Class in unique_values_arr_y:
    count_y_1 = count_y[Class] + noise_1
    prior = (count_y_1+1)/(len(train_df)+1)
    Prior_prob.append(prior)

  def Probability_NB(sample_no):
    prob_NB = []
    for i in range(len(unique_values_arr_y)) :
      prob = 1
      test_class = test_X.iloc[sample_no]
      A = test_class.to_numpy()
      att = []
      for k in range(len(test_class)):
          att.append(test_class[k])
          probability_att = prob_cond_num[k][np.where(unique_values_arr_X[k] == A[k])[0][0]][i]
          prob = probability_att*prob
      prob_NB.append(prob*Prior_prob[i])
    return prob_NB.index(max(prob_NB))

  test_y_arr = test_y.to_numpy()
  Count_p = 0
  for i in range(len(test_y)):
    if unique_values_arr_y[Probability_NB(i)] == test_y_arr[i]:
      Count_p += 1
  Accuracy = Count_p/len(test_y)*100
  return Accuracy

In [ ]:
Epsilon = [0.01,0.1,1]

In [ ]:
Accuracy_arr = [0.0  for _ in range(len(Epsilon))]
for i in range(len(Epsilon)):
  Accuracy_arr[i] = Accuracy_NB(Epsilon[i])
  
print(Accuracy_arr)

In [ ]:
import matplotlib.pyplot as plt
plt.gca().invert_xaxis()
plt.xscale("log")
plt.plot(Epsilon, Accuracy_arr, 'r-')     
plt.legend()                             
plt.xlabel('Epsilon')                          
plt.ylabel('Accuracy')                          
plt.title('Nursery dataset: 13K records, 8 attributes and 5 classes')         
plt.show()